In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "/kaggle/input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


/kaggle/input/individual-face-recognition/image/image/Data/Male/gsreas.16.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/rlocke.5.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/kjwith.1.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/dakram.7.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/9332898.18.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/cwchoi.1.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/jdbenm.4.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/jcarte.17.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/9338446.6.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/muthay.13.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/sidick.9.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/ggrego.10.jpg
/kaggle/input/individual-face-recognition/image/image/Data/Male/cjcar

In [2]:
import os
os.getcwd()

'/kaggle/working'

In [3]:
import tensorflow as tf 

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.utils import to_categorical
from PIL import Image 
from keras.models  import Model,Sequential
from keras.layers import Flatten,Dense, Dropout
from keras.layers import Convolution2D, MaxPool2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam 
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


In [5]:
import pandas as pd
import os
import cv2

In [6]:
os.getcwd()

'/kaggle/working'

In [7]:
PATH = ("/kaggle/input/individual-face-recognition/image/image")
#Test_Path = ("/kaggle/input/face-recognition/test/test")

In [8]:
DATA_PATH = os.path.join(PATH, 'Data')
data_dir_list = os.listdir(DATA_PATH)
print(data_dir_list)

['Male', 'Female']


In [9]:
img_rows=224
img_cols=224
num_channel=3

num_epoch = 15
batch_size = 32

img_data_list=[]
classes_names_list=[]
target_column=[]

In [10]:
for dataset in data_dir_list:
    classes_names_list.append(dataset)
    print("Getting images from {} folder\n".format(dataset))
    img_list = os.listdir(DATA_PATH +'/'+ dataset)
    for img in img_list:
        input_img = cv2.imread(DATA_PATH + '/' + dataset + '/' + img)
        input_img_resize=cv2.resize(input_img,(img_rows,img_cols))
        img_data_list.append(input_img_resize)
        target_column.append(dataset)

Getting images from Male folder

Getting images from Female folder



In [11]:
num_classes = len(classes_names_list)
print(num_classes)

2


In [12]:
#normalizing it 
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

In [13]:
#resizing the images and the num of samples
print(len(img_data))
print(img_data.shape)

2378
(2378, 224, 224, 3)


In [14]:
#what is happening?
classes_names_list

['Male', 'Female']

In [15]:
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

In [16]:
print(input_shape)

(224, 224, 3)


In [17]:
from sklearn.preprocessing import LabelEncoder
Labelencoder = LabelEncoder()
target_column = Labelencoder.fit_transform(target_column)

In [18]:
from collections import Counter
#Counter(classes).values()

In [19]:
classes = target_column
    # classes = np.ones((num_of_samples,),dtype = 'int64')

In [20]:
classes

array([1, 1, 1, ..., 0, 0, 0])

In [21]:
classes.shape

(2378,)

In [22]:
from keras.utils import to_categorical

classes = to_categorical(classes, num_classes)

In [23]:
from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=123)

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [25]:
y_test.shape

(476, 2)

In [26]:
y_test[0:5]

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [28]:
model = Sequential()

model.add(Conv2D(16,(3,3),activation = "relu", input_shape=input_shape))
model.add(Conv2D(16,(3,3),activation = "relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))


model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))

In [29]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 220, 220, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 110, 110, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 110, 110, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 193600)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                6195232   
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
__________

In [30]:
cnnfr = model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epoch, verbose=1, validation_data=(X_test, y_test))

Train on 1902 samples, validate on 476 samples
Epoch 1/15
1902/1902 [==============================] - 6s 3ms/step - loss: 0.3962 - acc: 0.8486 - val_loss: 0.1446 - val_acc: 0.9128
Epoch 2/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.1776 - acc: 0.9096 - val_loss: 0.0819 - val_acc: 0.9664
Epoch 3/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.1017 - acc: 0.9593 - val_loss: 0.0387 - val_acc: 0.9821
Epoch 4/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.0816 - acc: 0.9719 - val_loss: 0.0108 - val_acc: 0.9989
Epoch 5/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.0503 - acc: 0.9858 - val_loss: 0.0069 - val_acc: 1.0000
Epoch 6/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.0306 - acc: 0.9903 - val_loss: 0.0038 - val_acc: 0.9989
Epoch 7/15
1902/1902 [==============================] - 2s 1ms/step - loss: 0.0435 - acc: 0.9884 - val_loss: 0.0107 - val_acc: 0.9979
Epoch 8/15
1902

In [31]:
DATA_PATH_test = Test_Path
data_dir_list = os.listdir(DATA_PATH_test)
print(data_dir_list)

NameError: name 'Test_Path' is not defined